# Quick tour

This notebook showcases a easiest example of training neural network potentials with PiNN

In [ ]:
import tensorflow as tf
from pinn.datasets.qm9 import load_QM9_dataset
from pinn.models import potential_model
from pinn.calculator import PiNN_calc

## Getting the dataset

PiNN adapted tensorflow's dataset API to handel different datasets.

The dataset will be automatically splitted into subsets accoring to the split_ratio.  
Note that to use the dataset with the estimator, the datasets should be a function, instead of a dataset object.

In [ ]:
datasets = load_QM9_dataset('dsgn16/*', split_ratio={'train':8, 'test':2})
train = lambda: datasets['train']().repeat().batch(100)
test = lambda: datasets['test']().repeat().batch(10)

## Defining the model
In PiNN, models are defined at two levels: models and networks. 
- A model (model_fn) defines the target, loss and training details of the network
- A network defines the structure of the neural network  

In this exampel, we will use the potential model, and the PiNN network.

The configuration of a model is stored in a nested json-like structure. 

```Python
{'network':
    {'func': 'pinn_network',
     'params': 'ii_nodes':[10,10] ...}
 'training':
    {'learning_rate': 1e-5}
 ...}
```
A detailed description of the configuration can be found **NOWHERE**.

When using implemented network structures, the network function can be specified with a string.  
You can also provide a custom function here.

In [ ]:
params = {'network': {'func': 'pinn_network'}}
model = potential_model(params)

## Configuring the training process
The defined model is indeed a tf.Estimator object, thus, the training can be easily controlled

In [ ]:
train_spec = tf.estimator.TrainSpec(input_fn=train, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=100)

## Train and evaluate

In [ ]:
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

## Using the model

In [ ]:
calc = PiNN_calc(model)